In [ ]:

import csv
import os
import heapq
from tempfile import NamedTemporaryFile
import random

class ExternalMergeSort:
    def __init__(self, buffer_size=100000):
        self.BUFFER_SIZE = buffer_size
        self.temp_files = []

    def _sort_in_memory(self, block, key_idx, reverse):
        if len(block) <= 1:
            return block

        mid = len(block) // 2
        left = self._sort_in_memory(block[:mid], key_idx, reverse)
        right = self._sort_in_memory(block[mid:], key_idx, reverse)

        return self._merge(left, right, key_idx, reverse)

    def _merge(self, left, right, key_idx, reverse):
        result = []
        i = j = 0

        while i < len(left) and j < len(right):
            left_val = left[i][key_idx]
            right_val = right[j][key_idx]

            try:
                left_val = float(left_val) if '.' in left_val or left_val.isdigit() else left_val
                right_val = float(right_val) if '.' in right_val or right_val.isdigit() else right_val
            except ValueError:
                pass

            if (left_val <= right_val) if not reverse else (left_val >= right_val):
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        result.extend(left[i:])
        result.extend(right[j:])
        return result

    def _create_runs(self, input_file, key_idx, reverse):
        runs = []
        current_run = []
        current_size = 0

        with open(input_file, 'r') as f:
            reader = csv.reader(f)
            header = next(reader)

            for row in reader:
                row_size = sum(len(str(field)) for field in row)
                if current_size + row_size > self.BUFFER_SIZE and current_run:
                    sorted_run = self._sort_in_memory(current_run, key_idx, reverse)
                    temp_file = NamedTemporaryFile(mode='w+', delete=False, suffix='.csv')
                    with temp_file:
                        writer = csv.writer(temp_file)
                        writer.writerow(header)
                        writer.writerows(sorted_run)
                    runs.append(temp_file.name)
                    current_run = []
                    current_size = 0

                current_run.append(row)
                current_size += row_size

            if current_run:
                sorted_run = self._sort_in_memory(current_run, key_idx, reverse)
                temp_file = NamedTemporaryFile(mode='w+', delete=False, suffix='.csv')
                with temp_file:
                    writer = csv.writer(temp_file)
                    writer.writerow(header)
                    writer.writerows(sorted_run)
                runs.append(temp_file.name)

        return runs, header

    def _merge_runs(self, runs, header, key_idx, reverse, output_file):
        while len(runs) > 1:
            new_runs = []

            for i in range(0, len(runs), 2):
                if i + 1 < len(runs):
                    merged_run = self._merge_two_runs(runs[i], runs[i+1], header, key_idx, reverse)
                    new_runs.append(merged_run)
                    os.remove(runs[i])
                    os.remove(runs[i+1])
                else:
                    new_runs.append(runs[i])

            runs = new_runs

        if runs:
            with open(runs[0], 'r') as src, open(output_file, 'w') as dst:
                dst.write(src.read())
            os.remove(runs[0])

    def _merge_two_runs(self, run1, run2, header, key_idx, reverse):
        temp_file = NamedTemporaryFile(mode='w+', delete=False, suffix='.csv')

        with open(run1, 'r') as f1, open(run2, 'r') as f2, temp_file as out:
            writer = csv.writer(out)
            writer.writerow(header)

            reader1 = csv.reader(f1)
            reader2 = csv.reader(f2)
            next(reader1)
            next(reader2)

            row1 = next(reader1, None)
            row2 = next(reader2, None)

            while row1 is not None and row2 is not None:
                val1 = row1[key_idx]
                val2 = row2[key_idx]

                try:
                    val1 = float(val1) if '.' in val1 or val1.isdigit() else val1
                    val2 = float(val2) if '.' in val2 or val2.isdigit() else val2
                except ValueError:
                    pass

                if (val1 <= val2) if not reverse else (val1 >= val2):
                    writer.writerow(row1)
                    row1 = next(reader1, None)
                else:
                    writer.writerow(row2)
                    row2 = next(reader2, None)

            while row1 is not None:
                writer.writerow(row1)
                row1 = next(reader1, None)

            while row2 is not None:
                writer.writerow(row2)
                row2 = next(reader2, None)

        return temp_file.name

    def sort(self, input_file, output_file, key_column, order='asc'):
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Arquivo de entrada '{input_file}' nao encontrado")

        reverse = (order.lower() == 'desc')

        with open(input_file, 'r') as f:
            reader = csv.reader(f)
            header = next(reader)
            key_idx = header.index(key_column) if isinstance(key_column, str) else key_column

        runs, header = self._create_runs(input_file, key_idx, reverse)
        self._merge_runs(runs, header, key_idx, reverse, output_file)

def generate_test_file(filename, num_rows=10000):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'nome', 'valor'])
        used_ids = set()
        for i in range(num_rows):
            while True:
                new_id = random.randint(1, num_rows)
                if new_id not in used_ids:
                    used_ids.add(new_id)
                    break
            writer.writerow([
                new_id,
                f'Produto {i}',
                round(random.uniform(10.0, 1000.0), 2)
            ])

def verify_sorted_file(filename, key_column, order='asc'):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        try:
            key_idx = header.index(key_column) if isinstance(key_column, str) else key_column
        except ValueError:
            print(f"Erro: Coluna '{key_column}' nao encontrada no arquivo")
            print(f"Cabecalho disponivel: {header}")
            return False

        prev = None
        for i, row in enumerate(reader, 1):
            try:
                current = float(row[key_idx]) if '.' in row[key_idx] or row[key_idx].isdigit() else row[key_idx]
            except ValueError:
                current = row[key_idx]

            if prev is not None:
                if (order == 'asc' and current < prev) or (order == 'desc' and current > prev):
                    print(f"Erro na linha {i+1}:")
                    print(f"  Valor anterior: {prev} (linha {i})")
                    print(f"  Valor atual: {current} (linha {i+1})")
                    return False

            prev = current

        return True

if __name__ == "__main__":
    input_filename = "dados_teste.csv"
    output_filename = "dados_ordenados.csv"

    print("Gerando arquivo de teste com IDs unicos...")
    generate_test_file(input_filename, num_rows=1000)

    print("Iniciando ordenacao externa...")
    sorter = ExternalMergeSort(buffer_size=10000)
    sorter.sort(
        input_file=input_filename,
        output_file=output_filename,
        key_column="id",
        order="asc"
    )

    print("\nPrimeiras linhas do resultado:")
    with open(output_filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        print(header)
        for i, row in enumerate(reader):
            if i < 10:
                print(row)
            if i == 10:
                print("...")
                break

    print("\nVerificando ordenacao...")
    if verify_sorted_file(output_filename, "id", "asc"):
        print("Arquivo esta corretamente ordenado!")
    else:
        print("Erro: Arquivo nao esta ordenado corretamente!")

    os.remove(input_filename)
    os.remove(output_filename)

Gerando arquivo de teste com IDs unicos...
Iniciando ordenacao externa...

Primeiras linhas do resultado:
['id', 'nome', 'valor']
['1', 'Produto 138', '380.63']
['2', 'Produto 400', '982.3']
['3', 'Produto 872', '222.75']
['4', 'Produto 457', '750.51']
['5', 'Produto 903', '226.96']
['6', 'Produto 902', '664.09']
['7', 'Produto 434', '854.02']
['8', 'Produto 252', '744.78']
['9', 'Produto 634', '388.82']
['10', 'Produto 776', '732.62']
...

Verificando ordenacao...
Arquivo esta corretamente ordenado!
